# Commands

In [ ]:
# cd ~/twinews-logs ; jupython -o nohup-random-$HOSTNAME.out --venv st-venv ~/Workspace/Python/Datasets/Twinews/twinews/models/random.ipynb

# Init

In [ ]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
isNotebook = '__file__' not in locals()

In [ ]:
TEST = isNotebook # isNotebook, True, False

In [ ]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from nlptools.preprocessing import *
from nlptools.basics import *
from twinews.utils import *
from twinews.models.ranking import *
from machinelearning.iterator import *

In [ ]:
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [ ]:
from gensim.summarization import bm25

In [ ]:
logger = Logger(tmpDir('logs') + "/random.log") if isNotebook else Logger("random-" + getHostname() + ".log")
tt = TicToc(logger=logger)
tt.tic()

# Config

In [ ]:
# modelName = 'worst'
# modelName = 'ideal'
modelName = 'random'

In [ ]:
config = \
{
    'splitVersion': 2,
    'maxUsers': None if TEST else None, # Sub-sampling
    'seed': 0,
}

In [ ]:
if modelName != 'random':
    del config['seed']

In [ ]:
bp(config, logger, 5)

In [ ]:
# Here we check if we already generated ranking for this model with this specific config:
if not isNotebook:
    if rankingExists(modelName, config, logger=logger):
        raise Exception(modelName + " with this config already exist:\n" + b(config, 5))

# Getting data

In [ ]:
# Getting users and news
evalData = getEvalData(config['splitVersion'], maxExtraNews=0, maxUsers=config['maxUsers'], logger=logger)
(trainUsers, testUsers, trainNews, testNews, candidates, extraNews) = \
(evalData['trainUsers'], evalData['testUsers'], evalData['trainNews'],
 evalData['testNews'], evalData['candidates'], evalData['extraNews'])
bp(evalData.keys(), 5, logger)
log(b(evalData['meta'], 5), logger)

In [ ]:
# Here it is important to convert urls to lists because we want the same order to retrieve vectors by index...
# And we shuffle it so we do not stick urls a a user at the begin...
# But we seed the random to always have same order...
trainNewsList = shuffle(list(trainNews), seed=0)
testNewsList = shuffle(list(testNews), seed=0)
newsList = trainNewsList + testNewsList

In [ ]:
# Print all:
log(str(len(trainNewsList)) + " urls for trainNewsList", logger=logger)
log(str(len(testNewsList)) + " urls for testNewsList", logger=logger)
log(str(len(newsList)) + " urls for newsList", logger=logger)

In [ ]:
tt.tic("Data preprocessed")

# Ranking

In [ ]:
rankings = dict()
for userId in evalData["candidates"]:
    currentRankings = []
    for currentCandidates in evalData["candidates"][userId]:
        relevantUrls = set(evalData["testUsers"][userId].keys())
        unrelevantUrls = [e for e in currentCandidates if e not in relevantUrls]
        relevantUrls = sorted(list(relevantUrls))
        unrelevantUrls = sorted(list(unrelevantUrls))
        assert len(relevantUrls) > 0
        assert len(unrelevantUrls) > 0
        if modelName == 'worst':
            ranking = unrelevantUrls + relevantUrls
        elif modelName == 'ideal':
            ranking = relevantUrls + unrelevantUrls
        elif modelName == 'random':
            ranking = shuffle(relevantUrls + unrelevantUrls, seed=config['seed'])
        assert len(ranking) == len(currentCandidates)
        currentRankings.append(ranking)
    rankings[userId] = currentRankings
bp(rankings, logger)

In [ ]:
tt.tic("Rankings done")

# Adding rankings to the db

In [ ]:
# Read the doc!
addRanking(modelName, rankings, config, logger=logger)

In [ ]:
tt.tic("Rankings stored")

In [ ]:
totalDuration = tt.toc()

In [ ]:
notif(modelName + '-' + objectToHash(config)[:5] + " done in " + secondsToHumanReadableDuration(totalDuration) + " on " + getHostname())